In [6]:
%load_ext autoreload
%autoreload 2
from logistic import LogisticRegression#, GradientDescentOptimizer#, NewtonOptimizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import torch

In [34]:
X = torch.tensor([[1, 2, 3, 4], [1, 2, 3, 4]])
y = torch.tensor([1, 1, 1, 1])
X = X.type(torch.FloatTensor)
y = y.type(torch.FloatTensor)
LR = LogisticRegression()

In [40]:
X.size()[1]
LR.loss(X, y)
print(LR.grad(X, y))

tensor([-0.0020, -0.0040, -0.0060, -0.0080])


# Abstract

# Part A: Implement Logistic Regression

# Part B: Experiments

# Part C: Writing

# Conclusion